In [27]:
import numpy as np
import awkward as ak
import hist
import warnings
import pickle
from coffea.ml_tools.torch_wrapper import torch_wrapper
import hist
from sklearn.metrics import roc_curve, auc
import os
import json
import copy

In [11]:
warnings.filterwarnings('ignore', 'invalid value')
warnings.filterwarnings('ignore', 'No format')
warnings.filterwarnings('ignore', 'overflow encountered in cast')
warnings.filterwarnings('ignore', 'divide by zero encountered in divide')

In [12]:
hgg = ak.from_parquet('/scratch365/cmoore24/training/data/ecfs/hgg_ecfs.parquet')

In [13]:
qcd = ak.firsts(ak.from_parquet('/scratch365/cmoore24/training/data/ecfs/qcd_ecfs.parquet'))

In [14]:
hgg = hgg[ak.flatten(hgg.msoftdrop < 200)]
hgg = hgg[ak.flatten(hgg.msoftdrop > 40)]
hgg = hgg[ak.flatten(hgg.pt < 600)]
hgg = hgg[ak.flatten(hgg.pt > 450)]

In [15]:
qcd = qcd[qcd.msoftdrop < 200]
qcd = qcd[qcd.msoftdrop > 40]
qcd = qcd[qcd.pt < 600]
qcd = qcd[qcd.pt > 450]

In [16]:
mask = ak.ones_like(hgg[hgg.fields[0]], dtype='bool')
mask = ak.fill_none(mask, True)
for j in hgg.fields:
    if hgg[j].fields == []:
        mask = mask & (~ak.is_none(ak.nan_to_none(hgg[j])))
    else:
        for i in hgg[j].fields:
            mask = mask & (~ak.is_none(ak.nan_to_none(hgg[j][i])))
hgg = hgg[ak.flatten(mask)]

In [17]:
mask = ak.ones_like(qcd[qcd.fields[0]], dtype='bool')
mask = ak.fill_none(mask, True)
for j in qcd.fields:
    if qcd[j].fields == []:
        mask = mask & (~ak.is_none(ak.nan_to_none(qcd[j])))
    else:
        for i in qcd[j].fields:
            mask = mask & (~ak.is_none(ak.nan_to_none(qcd[j][i])))
qcd = qcd[mask]

In [18]:
hgg_train = ak.zip({
    'd2b1': hgg.d2b1,
    'ECFs': hgg.ECFs,
    'msoftdrop': hgg.msoftdrop,
    'pt': hgg.pt,
},
       depth_limit=1
      )

In [19]:
qcd_train = ak.zip({
    'd2b1': qcd.d2b1,
    'ECFs': qcd.ECFs,
    'msoftdrop': qcd.msoftdrop,
    'pt': qcd.pt,
},
       depth_limit=1
      )

In [21]:
with open('/scratch365/cmoore24/training/hgg/binary/ecfs_project/combo_training/combinations.pkl', 'rb') as f:
    combinations = pickle.load(f)

In [22]:
with open('../../jsons/subregion_event_totals.json', 'r') as f:
    totals = json.load(f)
with open('../../jsons/my_xsecs.json', 'r') as f:
    xsecs = json.load(f)

In [23]:
class EnergyCorrelatorFunctionTagger(torch_wrapper):
    def prepare_awkward(self, events, scaler, imap):
        fatjets = events
    

    
        retmap = {
            k: ak.concatenate([x[:, np.newaxis] for x in imap[k].values()], axis=1)
            for k in imap.keys()
        }
        x = ak.values_astype(scaler.transform(retmap['vars']), "float32")
        return (x,), {}

In [24]:
def get_cut(qcd_scores, break_val):
    hrange=(ak.min(qcd_scores), ak.max(qcd_scores))
    proportion=1.0
    i = 0
    while proportion > 0.15:
        qcd_hist = np.histogram(qcd_scores, bins=1000, 
                     range=hrange
                    )
        largest_bin_indices = np.argsort(qcd_hist[0])[-100:]
        largest_bin_vals = qcd_hist[1][largest_bin_indices]
        hrange = (largest_bin_vals[0], ak.max(qcd_scores))
        proportion = sum(qcd_hist[0])/len(qcd_scores)
        #print(proportion)
        i += 1
        if i > break_val:
            break
    cumulative_distribution = np.cumsum(qcd_hist[0][min(largest_bin_indices):max(largest_bin_indices)])
    total_count = cumulative_distribution[-1]
    half_count = total_count / 2
    median_bin_index = np.where(cumulative_distribution >= half_count)[0][0]
    cut = qcd_hist[1][median_bin_index]
    return cut

In [31]:
def imapper(array, ratio_list):
    imap = {}
    imap['vars'] = {}
    for i in ratio_list:
        imap['vars'][i] = array[i]
    return imap

In [32]:
def add_ratio(ratio, dataframe):
    dash = ratio.find('/')
    asterisk = ratio.find('*')
    numerator = ratio[:dash]
    denominator = ratio[dash+1:asterisk]
    exponent = float(ratio[asterisk+2:])
    num_ecf = dataframe[numerator]
    den_ecf = dataframe[denominator]
    ecf_ratio = (num_ecf)/(den_ecf**exponent)
    return ecf_ratio

In [35]:
for i in range(0,70):
    ratio_list = combinations[str(i)]
    model = f'/scratch365/cmoore24/training/hgg/binary/ecfs_project/combo_training/outputs/models/traced_model{i}.pt'
    scaler = f'/scratch365/cmoore24/training/hgg/binary/ecfs_project/combo_training/outputs/scalers/scaler{i}.pkl'
    with open(scaler, 'rb') as f:
        scaler = pickle.load(f)

    hgg_sub_array = copy.deepcopy(hgg_train)
    for j in ratio_list:
        hgg_sub_array[j] = add_ratio(j, hgg_sub_array.ECFs)
                    

    qcd_sub_array = copy.deepcopy(qcd_train)
    for j in ratio_list:
        qcd_sub_array[j] = add_ratio(j, qcd_sub_array.ECFs)

    tagger = EnergyCorrelatorFunctionTagger(model)
    
    hgg_imap = imapper(hgg_sub_array, ratio_list)
    hgg_scores = tagger(hgg_sub_array, scaler, hgg_imap)[:,0]

    qcd_imap = imapper(qcd_sub_array, ratio_list)
    qcd_scores = tagger(qcd_sub_array, scaler, qcd_imap)[:,0]

    nan_mask2 = np.isnan(hgg_scores)
    hgg_sub_array = hgg_sub_array[~nan_mask2]
    hgg_scores = hgg_scores[~nan_mask2]

    nan_mask2 = np.isnan(qcd_scores)
    qcd_sub_array = qcd_sub_array[~nan_mask2]
    qcd_scores = qcd_scores[~nan_mask2]

    bkg_zeros = ak.zeros_like(qcd_scores)
    sig_ones = ak.ones_like(hgg_scores)
    combined = ak.concatenate([qcd_scores,hgg_scores])
    combined_truth = ak.concatenate([bkg_zeros, sig_ones])

    try:
        fpr, tpr, thresholds = roc_curve(combined_truth, combined)
        roc_auc = auc(fpr, tpr)
    except:
        with open('ecf_results.json', 'r') as f:
            results = json.load(f)
    
        results[str(i)] = {'roc_auc': None, 'sculpt_metric': None}
    
        with open('ecf_results.json', 'w') as f:
            json.dump(results, f)
        continue

    if ak.min(qcd_scores) < -8:
        cut = get_cut(qcd_scores, 100)
    else:
        cut = get_cut(qcd_scores, 50)

    mask = ~((qcd_scores > cut))
    qcd_cut_msd = qcd_sub_array.msoftdrop[mask]
    qcd_fail_hist = hist.Hist.new.Reg(40, 40, 200, name='msd', label='QCD MSD').Weight()
    qcd_fail_hist.fill(msd=qcd_cut_msd);

    mask = ((qcd_scores > cut))
    qcd_cut_msd = qcd_sub_array.msoftdrop[mask]
    qcd_pass_hist = hist.Hist.new.Reg(40, 40, 200, name='msd', label='QCD MSD').Weight()
    qcd_pass_hist.fill(msd=qcd_cut_msd);

    scale = ((44.99*(xsecs['qcd']['qcd_470to600']*1000))/totals['qcd']['470to600'])
    qcd_pass_hist.view(flow=True)[:] *= scale
    qcd_fail_hist.view(flow=True)[:] *= scale

    total_qcd_hist = qcd_pass_hist + qcd_fail_hist

    sculpt_metric = sum(abs(total_qcd_hist.density() - qcd_pass_hist.density()))

    with open('ecf_results.json', 'r') as f:
        results = json.load(f)

    results[str(i)] = {'roc_auc': roc_auc, 'sculpt_metric': sculpt_metric}

    with open('ecf_results.json', 'w') as f:
        json.dump(results, f)